<a href="https://colab.research.google.com/github/arrafiiqbal/MBA/blob/main/MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIMPLE MARKET BASKET ANALYSIS USING APRIORI ALGORITHM

# 1. Import module and load the dataset

In [ ]:
#modules used in this project
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori

from google.colab import drive
drive.mount("/content/gdrive")

#dataset used
df=pd.read_excel(f'/content/gdrive/My Drive/FGA 2021/Online Retail.xlsx')

Mounted at /content/gdrive


# 2. Dataset Overview

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
print("There are",len(df),"transactions occured")
print("With",len(pd.unique(df['StockCode'])),"unique products")

There are 541909 transactions occured
With 4070 unique products


In [ ]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

Dataset contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

**Attribute Informations**

*   InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
*   StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. 
*  Description: Product (item) name. Nominal. 
* Quantity: The quantities of each product (item) per transaction. Numeric. 
* InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated. 
* UnitPrice: Unit price. Numeric, Product price per unit in sterling. 
* CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. 
* Country: Country name. Nominal, the name of the country where each customer resides. 


# 3. MARKET BASKET ANALYSIS

Market Basket Analysis is a technique that identifies the strength of association between a pair/group of products that are purchased together. In simpler words, it’s based on the concept that if a customer buys a product, they are more or less likely to buy another related product.

## 3.1 Data Pre-Processing

Changing datatypes into reasonable ones. Exclude cancelled transactions from df. Focus on UK country.

In [ ]:
#changing datatypes
df['InvoiceNo']=df['InvoiceNo'].astype('string')
df['StockCode']=df['StockCode'].astype('string')
df['Description']=df['Description'].astype('string')
df['CustomerID']=df['CustomerID'].astype('string')
df['Country']=df['Country'].astype('string')

df.dtypes

InvoiceNo              string
StockCode              string
Description            string
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             string
Country                string
dtype: object

In [ ]:
#filtering out cancelled transaction
df1=df[df['Country']=='United Kingdom']
df1=df1[df1['InvoiceNo'].str[0]!='c']

len(df1)

495478

In [ ]:
df1.describe()

,Quantity,UnitPrice
count,495478.000000,495478.000000
mean,8.605486,4.532422
std,227.588756,99.315438
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.100000
75%,10.000000,4.130000
max,80995.000000,38970.000000


The negative item quantity and price is unreasonable. Therefore, they should be filtered out.

In [ ]:
df1=df1[df1['Quantity']>0]
df1=df1[df1['UnitPrice']>0]

df1.describe()

,Quantity,UnitPrice,Revenue
count,485123.000000,485123.000000,485123.000000
mean,9.610738,3.849679,18.603987
std,161.904143,34.677371,281.280229
min,1.000000,0.001000,0.001000
25%,1.000000,1.250000,3.360000
50%,3.000000,2.100000,8.420000
75%,10.000000,4.130000,16.950000
max,80995.000000,13541.330000,168469.600000


Checking if there is any NaN in the column. And calculate how many

In [ ]:
for i in range(len(df1.columns)):
  print("There are",df1[df1.columns[i]].isnull().sum()," NaN values in",df1.columns[i],"column")

There are 0  NaN values in InvoiceNo column
There are 0  NaN values in StockCode column
There are 0  NaN values in Description column
There are 0  NaN values in Quantity column
There are 0  NaN values in InvoiceDate column
There are 0  NaN values in UnitPrice column
There are 130802  NaN values in CustomerID column
There are 0  NaN values in Country column
There are 0  NaN values in Revenue column


We can leave the dataframe as it is. NaN values in product desc is tolerable , moreover since the dataframe contains non-store online retail and the customer is mainly a wholesaler, NaN customer ID is also tolerable.

In [ ]:
print("The prepared dataset contains",len(df1),"transactions with",len(pd.unique(df1['StockCode'])),"unique products")

The prepared dataset contains 485123 transactions with 3916 unique products


The number of unique products is too many. We will foccus on Top 100 products with high revenue.

In [ ]:
#make column Revenue
df1['Revenue']=df1['Quantity']*df1['UnitPrice']

#revenue by product code sorted high to low
df1_sc=df1.groupby('StockCode').sum()
df1_sc=df1_sc.sort_values(['Revenue'],ascending=False)[:6]

In [ ]:
#0.8*df1_sc['Revenue'].sum()

In [ ]:
df1_sc

,Quantity,UnitPrice,Revenue
StockCode,,,
DOT,706,206248.77,206248.77
23843,80995,2.08,168469.60
22423,11068,24406.81,142273.29
85123A,35509,6784.69,98723.75
47566,16973,9442.43,93658.53
85099B,44264,4937.50,86471.34


In [ ]:
#206248.77+206248.77+206248.77+98723.75+93658.53+86471.34

In [ ]:
top80=df1_sc.index

In [ ]:
top80

Index(['DOT', '23843', '22423', '85123A', '47566', '85099B'], dtype='object', name='StockCode')

## 3.2 Transactions that Include the Top 100 product only

In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,15.3
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,15.3
177,536386,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 09:57:00,1.65,16029.0,United Kingdom,165.0
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom,163.2


In [ ]:
df1=df1.loc[df1['StockCode'].isin(top80)]

In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,15.3
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,15.3
177,536386,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 09:57:00,1.65,16029.0,United Kingdom,165.0
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom,163.2


Make the transactions into Basket (group by Invoice No)

In [ ]:
df1_basket=df1[['InvoiceNo','StockCode']]

In [ ]:
df1_basket['StockCode']=df1_basket.groupby('InvoiceNo')['StockCode'].transform(lambda x : ' '.join(x))
df1_basket=df1_basket.drop_duplicates()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1_basket.head()

,InvoiceNo,StockCode
0,536365,85123A
49,536373,85123A
66,536375,85123A
177,536386,85099B
220,536390,85123A 85099B


basket with multiple items only

In [ ]:
df1_basket['StockCode']=df1_basket['StockCode'].str.split()

In [ ]:
df1_basket['Items']=df1_basket['StockCode'].str.len()

In [ ]:
df1_basket=df1_basket[df1_basket['Items']>1]

In [ ]:
len(df1_basket)

1689

In [ ]:
df1_basket.head()

,InvoiceNo,StockCode,Items
220,536390,"[85123A, 85099B]",2
935,536502,"[85123A, 22423]",2
1429,536542,"[85099B, 85123A]",2
1576,536544,"[22423, 85099B, 85123A, DOT]",4
2305,536575,"[85099B, 85123A]",2


Take StockCode column for MBA analysis

In [ ]:
df_mba=df1_basket['StockCode']

In [ ]:
df_mba=pd.DataFrame(df_mba)
df_mba.head()

,StockCode
220,"[85123A, 85099B]"
935,"[85123A, 22423]"
1429,"[85099B, 85123A]"
1576,"[22423, 85099B, 85123A, DOT]"
2305,"[85099B, 85123A]"


## 3.3 Apiori Algorithm

In [ ]:
df_mba['StockCode']=df_mba['StockCode'].transform(lambda x : ' '.join(x))

In [ ]:
final = (pd.get_dummies(df_mba['StockCode'].str.split(expand=True))
         .groupby(lambda x: x.split('_')[-1],axis=1).sum().astype('bool'))

In [ ]:
final

,22423,47566,85099B,85123A,DOT
220,False,False,True,True,False
935,True,False,False,True,False
1429,False,False,True,True,False
1576,True,False,True,True,True
2305,False,False,True,True,False
...,...,...,...,...,...
539891,True,False,False,False,False
540216,True,False,False,True,False
540534,False,True,False,True,True
541290,True,True,False,False,False


In [ ]:
results=apriori(final, min_support=0.2, use_colnames=True)
results

,support,itemsets
0,0.426288,(22423)
1,0.488455,(47566)
2,0.558911,(85099B)
3,0.505625,(85123A)
4,0.357016,(DOT)
5,0.215512,"(22423, 47566)"
6,0.224393,"(85123A, 47566)"
7,0.229722,"(85123A, 85099B)"
8,0.287152,"(DOT, 85099B)"


In [ ]:
top80

Index(['DOT', '23843', '22423', '85123A', '47566', '85099B'], dtype='object', name='StockCode')

In [ ]:
#to increase sale, we can bundle 85123A with 47566 or 8509B

In [ ]:
df1[df1['StockCode']=='47566'][0]

NameError: ignored